In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, GlobalAveragePooling3D, Multiply, Reshape
from keras.optimizers import Adam
import nibabel as nib

def channel_attention(input_tensor, reduction_ratio=8):
    channel_axis = -1
    channels = input_tensor.shape[channel_axis]
    shared_dense_one = Dense(channels // reduction_ratio, activation='relu', use_bias=False)
    shared_dense_two = Dense(channels, activation='sigmoid', use_bias=False)
    avg_pool = GlobalAveragePooling3D()(input_tensor)
    avg_pool = Reshape((1, 1, 1, channels))(avg_pool)
    avg_pool = shared_dense_one(avg_pool)
    avg_pool = shared_dense_two(avg_pool)

    attention_output = Multiply()([input_tensor, avg_pool])
    return attention_output

def create_3d_cnn(input_shape, learning_rate, beta1, beta2):
    inputs = Input(shape=input_shape)

    x = Conv3D(32, (3, 3, 3), activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = Conv3D(64, (3, 3, 3), activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = Conv3D(128, (3, 3, 3), activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = channel_attention(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='linear')(x)

    model = Model(inputs, outputs)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta1, beta_2=beta2)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

    return model

In [ ]:
import os
import numpy as np
from scipy.ndimage import zoom

def resize_3d_image(image, target_shape=None, scale=None, order=1):
    if scale is None and target_shape is None:
        raise ValueError("Either target_shape or scale must be provided.")

    if scale is None:
        scale = [t / s for t, s in zip(target_shape, image.shape[:3])]

    if image.ndim == 4:
        resized = zoom(image, scale + [1], order=order)
    else:
        resized = zoom(image, scale, order=order)

    return resized

path = "/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/PPMI/PPMI_T1_CN_Preprocessed/PPMI"
list = []
eep = set()
image_dict2 = {}
count = 0
for (root, dirs, file) in os.walk(path):
    for f in file:
        if 'T2' in f:
            continue
        elif '.nii' in f:
            #print(f)
            path1 = root + '/' + f
            nifti_image = nib.load(path1)
            image_data = nifti_image.get_fdata()
            max_value = np.max(image_data)
            image_data = image_data / max_value
            image_data = resize_3d_image(image_data,target_shape=(64, 64, 64, 1))
            image_dict2[f[5:9]] = image_data
            count += 1
print(count)

In [ ]:
directory = '/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/HCP/'
files = os.listdir(directory)
files2 = []
for i in files:
    if 'nii' in i:
        files2.append(i)


image_dict = {}
count = 0
for j in files2:
    if j != '.nii.gz' and count < 300:
        path1 = directory + j
        nifti_image = nib.load(path1)
        image_data = nifti_image.get_fdata()
        max_value = np.max(image_data)
        image_data = image_data / max_value
        image_data = resize_3d_image(image_data,target_shape=(64, 64, 64, 1))
        image_dict[j[0:6]] = image_data
        count += 1

#path1 = '/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/HCP/100206_T1w.nii.gz'

#print(shapes)
nifti_image = nib.load(path1)
image_data = nifti_image.get_fdata()
print("Shape of the NIfTI image:", image_data.shape)

slice_index = image_data.shape[2] // 2
slice_data = image_data[:, :, slice_index]


In [4]:
import pandas as pd

df = pd.read_csv('/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/HCP/age/ages.csv')

ages = df['age'].tolist()
ids = df['subject_id'].tolist()
key = {}
for i,j in zip(ids,ages):
    key[i] = j

In [5]:
df2 = pd.read_csv('/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/PPMI/PPMI_T1_CN_Preprocessed_3_30_2025.csv')

ages = df2['Age'].tolist()
ids = df2['Subject'].tolist()
key2 = {}
for i,j in zip(ids,ages):
    key2[i] = j

In [ ]:
input_shape = model.input_shape  # Example: (None, 128, 128, 128, 1)
print("Model expects input shape:", input_shape)

In [ ]:
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint
input_shape = (64, 64, 64, 1)
#model.summary()
x_train = []
y_train = []
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for i in df['subject_id']:
    item = image_dict.get(str(i))
    if item is not None:
        x_train.append(item)
        y_train.append(float(key.get(i)))
del image_dict

for i in df2['Subject']:
    item = image_dict2.get(str(i))
    if item is not None:
        x_train.append(item)
        y_train.append(float(key2.get(i)))
del image_dict2

#print(y_train)
x_train = tf.convert_to_tensor(x_train, dtype=tf.float16)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float16)
lr = [1e-2, 1e-3, 1e-4]
beta_one = [.9, .99, .999]
beta_two = [.9, .99, .999]
#print(y_train)
count = 1
for fold, (train_idx, test_idx) in enumerate(kf.split(x_train)):
    for b1 in beta_one:
        for b2 in beta_two:
            for learn in lr:
                model_name = 'model' + str(count) + '_learn_' + str(learn) + '_beta1_' + str(b1) + '_beta2_' + str(b2)
                checkpoint = ModelCheckpoint(
                filepath= model_name + '.keras',       
                monitor='mae',            
                save_best_only=True,             
                mode='min',                     
                verbose=1                      
            )
                x_t = tf.gather(x_train,train_idx)
                x_val = tf.gather(x_train,test_idx)
                y_t = tf.gather(y_train,train_idx)
                y_val = tf.gather(y_train,test_idx)
                model = create_3d_cnn(input_shape,learning_rate=learn,beta1=b1,beta2=b2)
                history = model.fit(
                        x_t, y_t,
                        epochs=50,
                        batch_size=8,
                        validation_data=(x_val, y_val),
                        callbacks=[checkpoint]
                        )
                for i in range(len(history.history['loss'])):
                    train_loss = history.history['loss'][i]
                    val_loss = history.history['val_loss'][i]
                    #print(f"Epoch {i+1} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
                count += 1

In [ ]:
from keras.models import load_model

model = load_model('/projectnb/ec523/projects/proj_GS_LQ_EPB/models/Regression CNN Model T1W/best_model/model35_learn_0.001_beta1_0.9_beta2_0.999.keras')

In [ ]:
import numpy as np
import nibabel as nib
import os
from scipy.ndimage import zoom

def resize_3d_image(image, target_shape=None, scale=None, order=1):
    if scale is None and target_shape is None:
        raise ValueError("Either target_shape or scale must be provided.")

    if scale is None:
        scale = [t / s for t, s in zip(target_shape, image.shape[:3])]

    if image.ndim == 4:
        resized = zoom(image, scale + [1], order=order)
    else:
        resized = zoom(image, scale, order=order)

    return resized
input_shape = model.input_shape 
print("Model expects input shape:", input_shape)
target_shape = input_shape[1:4]
path1 = '/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/ADNI/ADNI/136_S_1227/MPR____N3__Scaled/2007-02-21_13_12_42.0/I66824'
#path1 = '/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/ADNI/ADNI/051_S_1072/MPR-R__GradWarp__B1_Correction__N3__Scaled/2007-01-29_11_13_30.0/I65662'
files = os.listdir(path1)
path1 = os.path.join(path1, files[0])
nifti_image = nib.load(path1)
image_data = nifti_image.get_fdata()
max_value = np.max(image_data)
image_data = image_data / max_value
image_data = resize_3d_image(image_data, target_shape)
image_data = np.expand_dims(image_data, axis=-1)  
image_data = np.expand_dims(image_data, axis=0) 
print("Final input shape:", image_data.shape)
prediction = model.predict(image_data)
print("Prediction:", prediction[0][0])